In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_df = pd.read_csv('../data/budget_df_cleaned1.csv', index_col=0)

In [18]:
budget_df.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,combined_text
0,Function__Teacher Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__NO_LABEL,Position_Type__Teacher,Object_Type__NO_LABEL,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,,...,,1.000000,,,kindergarten,50471.810,kindergarten,general fund,,teacher elementary kindergar...
1,Function__NO_LABEL,Use__NO_LABEL,Sharing__NO_LABEL,Reporting__NO_LABEL,Student_Type__NO_LABEL,Position_Type__NO_LABEL,Object_Type__NO_LABEL,Pre_K__NO_LABEL,Operating_Status__Non-Operating,contractor service,...,,0.420663,rgn gob,,undesignated,3477.860,building improvement service,,building improvement service,contractor service bond expenditure building f...
2,Function__Teacher Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Teacher,Object_Type__Base Salary/Compensation,Pre_K__Non PreK,Operating_Status__PreK-12 Operating,personal service teacher,...,,1.000000,,,teacher,62237.130,instruction regular,general purpose school,,personal service teacher tcher 2nd grade ...
3,Function__Substitute Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Substitute,Object_Type__Benefits,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,employee benefit,...,,0.420663,unalloc budget school,,professional instructional,22.300,general middle junior high sch,,regular instruction,employee benefit teacher sub general fund teac...
4,Function__Substitute Compensation,Use__Instruction,Sharing__School Reported,Reporting__School,Student_Type__Unspecified,Position_Type__Teacher,Object_Type__Substitute Compensation,Pre_K__NO_LABEL,Operating_Status__PreK-12 Operating,teacher coverage teacher,...,,0.420663,non project,,professional instructional,54.166,general high school education,,regular instruction,teacher coverage teacher teacher sub general f...


In [4]:
for col in budget_df[word_predictors]:
    budget_df[col] = budget_df[word_predictors].loc[:,[col]].fillna(' ')

In [5]:
def create_tfidf(word_column):
    tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)
    col_tfidf = tfidf.fit_transform(budget_df[word_column])
    
    return col_tfidf

In [88]:
Object_Description_tfidf = create_tfidf('Object_Description')
Text_2_tfidf= create_tfidf('Text_2')
SubFund_Description_tfidf = create_tfidf('SubFund_Description')
Job_Title_Description_tfidf = create_tfidf('Job_Title_Description')
Text_3_tfidf = create_tfidf('Text_3')
Text_4_tfidf= create_tfidf('Text_4')
Sub_Object_Description_tfidf= create_tfidf('Sub_Object_Description')
Location_Description_tfidf= create_tfidf('Location_Description')
Function_Description_tfidf= create_tfidf('Function_Description')
Facility_or_Department_tfidf= create_tfidf('Facility_or_Department')
Position_Extra_tfidf= create_tfidf('Position_Extra')
Program_Description_tfidf= create_tfidf('Program_Description')
Fund_Description_tfidf= create_tfidf('Fund_Description')
Text_1_tfidf= create_tfidf('Text_1')

In [92]:
len(word_predictors), len(tfidf_list)

(15, 15)

In [93]:
# word_predictors.append('combined_tfidif')

In [7]:
# combined_text_tfidf= create_tfidf('combined_text')

# combined_text_tfidf.get_shape()

In [91]:
tfidf_list = [Object_Description_tfidf, Text_2_tfidf, SubFund_Description_tfidf, Job_Title_Description_tfidf, Text_3_tfidf,\
              Text_4_tfidf,Sub_Object_Description_tfidf, Location_Description_tfidf, Function_Description_tfidf,\
              Facility_or_Department_tfidf, Position_Extra_tfidf, Program_Description_tfidf,\
              Fund_Description_tfidf, Text_1_tfidf, combined_tfidf]

In [77]:
from scipy import sparse
from scipy.sparse import hstack

combined_tfidf = hstack([Object_Description_tfidf, Text_2_tfidf, SubFund_Description_tfidf, Job_Title_Description_tfidf,\
                         Text_3_tfidf, Text_4_tfidf,Sub_Object_Description_tfidf, Location_Description_tfidf,\
                         Function_Description_tfidf,Facility_or_Department_tfidf, Position_Extra_tfidf,\ 
                         Program_Description_tfidf, Fund_Description_tfidf, Text_1_tfidf])

In [11]:
budget_tfidf.get_shape()

(400277, 4717)

## Model Predictions for Operating_status

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [95]:
def nlp_pipe(target_name, predictor_name, X, y):
     
    X_train, X_val, y_train, y_val = train_test_split(X, 
                                                    y, 
                                                    test_size=.2, 
                                                    random_state=42)
    
    for predictor in word_predictors:
        precitor_name = predictor
        
    nlp_pipe = Pipeline([
    ('clf', LogisticRegression(random_state=42, multi_class='ovr'))])

    
    model = nlp_pipe.fit(X_train, y_train)
    
    results = {'target': target_name,
               'predictor': predictor_name,
               'train_score': nlp_pipe.score(X_train, y_train),
               'val_score': nlp_pipe.score(X_val, y_val)}
    
    return model, results

In [96]:
op_status_results = []
for tfidf, predictor in zip(tfidf_list, word_predictors):  
    results = nlp_pipe('Operating Status', predictor, tfidf, le.fit_transform(budget_df['Operating_Status']))
    op_status_results.append(results)
    
       y = le.fit_transform(budget_df[target])
#     X = tfidf
#     results = nlp_pipe(X, y, target)
#     model_results.append(results)

In [99]:
op_status_results

[(Pipeline(steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]),
  {'predictor': 'Object_Description',
   'target': 'Operating Status',
   'train_score': 0.91974917322723992,
   'val_score': 0.91823223743379634}),
 (Pipeline(steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]),
  {'predictor': 'Text_2',
   'target': 'Operating Status',
   'train_score': 0.87587634789723345,
   'val_score': 0.8744378934745678}),
 (Pipeline(steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            in

In [117]:
score_list = []
for item in op_status_results:
    scores = item[1]
    score_list.append(scores)

In [119]:
pd.DataFrame(score_list)

,predictor,target,train_score,val_score
0,Object_Description,Operating Status,0.919749,0.918232
1,Text_2,Operating Status,0.875876,0.874438
2,SubFund_Description,Operating Status,0.934002,0.932909
3,Job_Title_Description,Operating Status,0.865412,0.864245
4,Text_3,Operating Status,0.858560,0.857512
5,Text_4,Operating Status,0.858798,0.857862
6,Sub_Object_Description,Operating Status,0.920030,0.918994
7,Location_Description,Operating Status,0.878212,0.876924
8,Function_Description,Operating Status,0.942034,0.940429
9,Facility_or_Department,Operating Status,0.859519,0.858586


In [122]:
import itertools

In [125]:
model_results = []
for target, (tfidf, predictor) in itertools.product(targets, zip(tfidf_list, word_predictors)):
    X = tfidf
    y = budget_df[target]
    results = nlp_pipe(target, predictor, X, y)
    model_results.append(results)
    
#        y = le.fit_transform(budget_df[target])
#     X = tfidf
#     results = nlp_pipe(X, y, target)
#     model_results.append(results)

In [126]:
model_results

[(Pipeline(steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]),
  {'predictor': 'Object_Description',
   'target': 'Function',
   'train_score': 0.53963044272549265,
   'val_score': 0.53641201159188567}),
 (Pipeline(steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]),
  {'predictor': 'Text_2',
   'target': 'Function',
   'train_score': 0.36372380324838161,
   'val_score': 0.35866143699410413}),
 (Pipeline(steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling

In [68]:
# import itertools

# model_results = []
# for tfidf, target in itertools.product(tfidf_list, targets):  
#     y = le.fit_transform(budget_df[target])
#     X = tfidf
#     results = nlp_pipe(X, y, target)
#     model_results.append(results)
    

In [120]:
X = combined_tfidf #need to do combined tfidf for test set
y = le.fit_transform(budget_df['Operating_Status'])
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state = 42)

In [ ]:
op_status_results[14][0].predict(X_val)
p_status_results[14][0].predict_proba(X_val)
list(le.classes_)
list(le.inverse_transform([0, 1, 2]))

## Figure out how to combine model of categorical columns and numerical

In [ ]:
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                     non_negative=True, norm=None, binary=False,
                                                     ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

### running into memory issue for get_dummies for the 9 cat